In [2]:
import sys
sys.path.insert(0, '../')
import numpy as np
np.random.seed(314)
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from FFNN import FFNN
from functions import *

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def softmax_derivative(z):
    return np.ones_like(z)

def one_hot_encode(y, num_classes=10):
    n = len(y)
    one_hot = np.zeros((n, num_classes))
    one_hot[np.arange(n), y.astype(int)] = 1
    return one_hot

def accuracy_score(y_true, y_pred):
    return np.mean(y_true == y_pred)

mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
X = mnist.data / 255.0
y = mnist.target.astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train_onehot = one_hot_encode(y_train)
y_test_onehot = one_hot_encode(y_test)

activations = [sigmoid, softmax]
derivatives = [sigmoid_derivative, softmax_derivative]

model = FFNN(784, [64, 10])

epochs = 10
batch_size = 32

for epoch in range(epochs):
    indices = np.arange(len(X_train))
    np.random.shuffle(indices)
    
    for i in range(0, len(X_train), batch_size):
        batch_indices = indices[i:i+batch_size]
        X_batch = X_train[batch_indices]
        y_batch = y_train_onehot[batch_indices]
        
        output = model.forward_pass(X_batch, activations)
        model.backward_pass(y_batch, derivatives, learning_rate=0.1)
    
    train_output = model.forward_pass(X_train, activations)
    train_pred = np.argmax(train_output, axis=1)
    train_acc = accuracy_score(y_train, train_pred)
    
    test_output = model.forward_pass(X_test, activations)
    test_pred = np.argmax(test_output, axis=1)
    test_acc = accuracy_score(y_test, test_pred)
    
    print(f"Epoch {epoch+1}: Train Acc = {train_acc:.4f}, Test Acc = {test_acc:.4f}")

Epoch 1: Train Acc = 0.8586, Test Acc = 0.8543
Epoch 2: Train Acc = 0.8904, Test Acc = 0.8879
Epoch 3: Train Acc = 0.9022, Test Acc = 0.8959
Epoch 4: Train Acc = 0.9134, Test Acc = 0.9040
Epoch 5: Train Acc = 0.9228, Test Acc = 0.9116
Epoch 6: Train Acc = 0.9286, Test Acc = 0.9163
Epoch 7: Train Acc = 0.9328, Test Acc = 0.9201
Epoch 8: Train Acc = 0.9371, Test Acc = 0.9234
Epoch 9: Train Acc = 0.9374, Test Acc = 0.9223
Epoch 10: Train Acc = 0.9421, Test Acc = 0.9265
